In [7]:
from zipfile import ZipFile
from tqdm import tqdm
import glob, re, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from pdb import set_trace as breakpoint # like in 3.7
from collections import Counter

In [95]:
with ZipFile('./data/court2018.zip') as zf:
    decisions = zf.namelist()

In [96]:
def split_paragraphs(doc):
    '''
    breaks text into paragraphs. also strips indents with whitespace
    '''
    # split and strip
    doc = map(str.strip, doc.split('\n'))
    # delete empty lines - result of multiple newlines
    doc = filter(lambda paragraph: len(paragraph) > 0,
                 doc)
    # remove duplicate whitespace
    return list(map(lambda paragraph: re.sub('\s+', ' ', paragraph),
                    doc))

## Which articles are used in the decision?

1. Filter list of paragraphs to leave only ones mentioning article

In [97]:
[p for p in paragraphs
 if re.search('\d{1,4} *ст[\. ]|ст\. *ст\. *\d{1,4} *- *\d{1,4}', p)]

['СВ Царичанського ВП Новомосковського ВП ГУНП в Дніпропетровській області проводиться досудове розслідування у кримінальному провадженні №1201804060000253 від 19.05.2018р. за ознаками вчинення злочину, передбаченого ч. 1 ст.115 КК України.',
 'Таким чином, ОСОБА_4, підозрюється в умисному вбивстві, тобто умисному протиправному заподіянні смерті ОСОБА_3, тобто у вчиненні злочину, передбаченого ч. 1 ст. 115 КК України.',
 '25.05.2018 року по вказаному кримінальному провадженню ОСОБА_4 було оголошено про підозру у вчиненні ним злочину, передбаченого ч. 1 ст. 115 КК України.',
 "Згідно ч. 1 ст. 242 КПК України, експертиза проводиться експертною установою, експертом або експертами, за дорученням слідчого судді чи суду, наданим за клопотанням сторони кримінального провадження або, якщо для з'ясування обставин, що мають значення для кримінального провадження необхідні спеціальні знання."]

### КК or КПК?
2. Rule-based approach = suffering, but often it is the simpliest and most efficient

In [98]:
df = []
article_freq = Counter()

with ZipFile('./data/court2018.zip') as zf:
    for fn in tqdm(decisions):
        with zf.open(fn) as f:
            
            category = re.search('\d+_(\d{4}).txt', fn).group(1)
            doc = f.read().decode()
            paragraphs = split_paragraphs(doc)
            
            doc_articles = set()
            
            for p in paragraphs:
                if re.search('\d{1,4} *ст[\. ]|ст\. *ст\. *\d{1,4} *- *\d{1,4}', p):
                    found = re.findall('ст[\. ]*(\d{1,4})| ст\. *ст\. *(\d{1,4} *- *\d{1,4})', p)
                    matches = set()
                    [matches.update(list(match)) for match in found]
                    matches.discard('')
                    
                    # Append the code of law of article
                    # it is always after the No of article
                    for article in list(matches):
                        position = re.search(f'ст[\. ]*{article}', p
                                    ).span()[-1]
                        ccu = re.search('кк укра[їи]| крим[іи]нал\w+ +кодекс',
                                        p.lower()[position: ])
                        cpcu = re.search('кпк укра[їи]| крим[іи]нал\w+[\- ]*процесуа\w+ +кодекс',
                                         p.lower()[position: ])
                        if ccu and not cpcu:
                            code = 'cc'
                        elif not ccu and cpcu:
                            code = 'cpc'
                        elif ccu and cpcu:
                            ccu_pos = ccu.span()[0]
                            cpcu_pos = cpcu.span()[0]
                            code = 'cpc' if cpcu_pos < ccu_pos else 'cc'
                        else:
                            code = 'other'
                        matches.update([(article, code)])
                        matches.discard(article)
                        
                    doc_articles.update(matches)
            
            for article, code in list(doc_articles):
                if '-' in article:
                    doc_articles.remove((article, code))
                    a1, a2 = article.split('-')
                    doc_articles.update([(str(a), code) for a in range(int(a1), int(a2) + 1)])
                    
            doc_articles = [f'{a}_{code}' for a, code in doc_articles]
                    
            data = {'fn': fn, 'category': category}
            for a in doc_articles:
                data[a] = 1
                
            df.append(data)
            article_freq.update(doc_articles)
df = pd.DataFrame(df)

100%|██████████| 18025/18025 [00:19<00:00, 904.18it/s]


In [79]:
article_freq.most_common(15)

[('115_cc', 11426),
 ('15_cc', 2330),
 ('177_cpc', 2257),
 ('331_cpc', 1742),
 ('187_cc', 1706),
 ('244_cpc', 1429),
 ('242_cpc', 1419),
 ('185_cc', 1298),
 ('314_cpc', 1207),
 ('263_cc', 1076),
 ('183_cpc', 929),
 ('243_cpc', 846),
 ('336_cpc', 805),
 ('178_cpc', 800),
 ('107_cpc', 742)]

### replace not frequent articles

In [99]:
columns_leave = [a for a, count in article_freq.most_common(10)]
columns_leave = ['fn', 'category'] + columns_leave
df = df.reindex(columns_leave, axis=1)
df = df.fillna(0)
# only ones that have at leat 1 article
df = df.loc[df['115_cc'] > 0].copy()

## Build a classifier 

### Filter data:
1. decisions on murder with robbery
1. without robbery (cc)

In [108]:
df['is_robbery'] = (df['187_cc'] > 0) | (df['185_cc'] > 0)
df.is_robbery.value_counts()

False    11127
True      3117
Name: is_robbery, dtype: int64

### Define train and test split

In [115]:
# shuffle rows
df = df.sample(frac=1)
test_point = int(len(df) * 0.2)
df_test = df.iloc[ :test_point].copy()
df_train = df.iloc[test_point: ].copy()
# df_test.to_csv('data/test_files_labels.csv', index=False)
# df_train.to_csv('data/train_files_labels.csv', index=False)

### Мішок слів
* Простиq підхід. Припускає, що зміст документа відображають його слова.
* Лише слова відображають зміст документа. *Слова документа відображають зміст лише.*
* Порядок, послідовності не мають значення

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from polyglot.text import Text
from zipfile import ZipFile
from tqdm import tqdm
from utils.preprocessing import *
from pdb import set_trace as breakpoint # like in 3.7
import re, pymorphy2, pandas as pd, numpy as np

In [2]:
vect = CountVectorizer()
look_here = vect.fit_transform(['Цьогорічний пляжний сезон засвідчив, що херсонці адаптувалися до нової реальності й після окупації Криму навчилися більше заробляти на туризмі. Скадовськ поки не Анталія, але потенціал є. Шансом і викликом водночас для регіону є децентралізація. Успішні громади організовують курорти, зводять парки, облаштовують освітлення. Про те, чим живуть, на що сподіваються та чого остерігаються жителі південного краю, розповідають Сергій Данилов, експерт Центру близькосхідних досліджень, та соціолог Віталій Юрасов, які вивчали регіон',
                                'Блакитний колір – показник у нормі, рожевий – забруднення перевищує допустиму межу. Відтепер можна легко відстежувати стан води у міських річках. На онлайн-карті "Чиста вода" Текстів і Державного агентства водних ресурсів відображені найбільші річкові басейни України та вказаний рівень забрудненості. Зауважимо, що йдеться не про русла річок повністю: на карті показано стан води у локаціях, де беруть проби для аналізів.',
                               ])
look_here = pd.DataFrame(look_here.toarray())
look_here.columns = vect.get_feature_names()
print(len(look_here.columns), 'words')
look_here

107 words


,агентства,адаптувалися,але,аналізів,анталія,басейни,беруть,блакитний,близькосхідних,більше,...,херсонці,центру,цьогорічний,чим,чиста,чого,шансом,що,юрасов,які
0,0,1,1,0,1,0,0,0,1,1,...,1,1,1,1,0,1,1,2,1,1
1,1,0,0,1,0,1,1,1,0,0,...,0,0,0,0,1,0,0,1,0,0


In [3]:
df_test= pd.read_csv('data/test_files_labels.csv')
df_train= pd.read_csv('data/train_files_labels.csv')

In [4]:
def tokenizer(text):
    '''
    split text into separate units - tokens. Words, punctuation, etc
    '''
    text = Text(text)
    return [str(word) for word in text.words]

**Tf-Idf** - не просто рахує слова. <br>Надає високе значення для слова в документі, якщо воно не дуже часто зустрічається у корпусі, проте відчутно частотніше в конкретному документі. <br> Слово "що" - всюди, що воно нам може сказати про документ? А слово "журналіст" вже вказує нам на зміст.

In [5]:
bow_vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                                 ngram_range=(1, 2),
                                 max_df=0.95, min_df=3)

Often we can't place all the data in memory. Generators are to help - 1 doc at a time

In [6]:
def traint_text_generator(df):
    with ZipFile('./data/court2018.zip') as zf:
        # never do nlp without progress bars - tqdm is a good one
        for filename in tqdm(df.fn.values):
            with zf.open(filename) as f:
                yield f.read().decode()

In [7]:
train_bow = bow_vectorizer.fit_transform(traint_text_generator(df_train))
train_bow

100%|██████████| 11396/11396 [01:45<00:00, 107.76it/s]


<11396x279289 sparse matrix of type '<class 'numpy.float64'>'
	with 11107337 stored elements in Compressed Sparse Row format>

![General steps](assets/meme.png)

Our ML data:
- **X** - features/variables/some ugly large matrix/data
- **y** - labels - is the decision about robbery

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

regression = LogisticRegression()
regression.fit(X=train_bow,
               y=df_train.is_robbery.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Time to see the metrics - **evaluate model**

In [14]:
test_bow = bow_vectorizer.transform(traint_text_generator(df_test))

y_true = df_test.is_robbery.values
y_pred = regression.predict(test_bow)

100%|██████████| 2848/2848 [00:22<00:00, 127.43it/s]


In [19]:
print(classification_report(y_true=y_true, y_pred=y_pred, labels=[True, False]))

             precision    recall  f1-score   support

       True       0.99      0.78      0.87       638
      False       0.94      1.00      0.97      2210

avg / total       0.95      0.95      0.95      2848



In [20]:
confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[True, False])

array([[ 499,  139],
       [   7, 2203]])

From wikipedia:<br>
**precision** (also called positive predictive value) is the fraction of relevant instances among the retrieved instances
<br>**recall** (also known as sensitivity) is the fraction of relevant instances that have been retrieved over the total amount of relevant instances.

![Precision & recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/440px-Precisionrecall.svg.png)